# Implementing CNN Using Pytorch As Neural Network API

## Let's kick things off with our imports: 

In [1]:

from torchvision import datasets,transforms
import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import confusion_matrix as c
import matplotlib.pyplot as plt
from itertools import product
torch.set_grad_enabled(True)
torch.set_printoptions(linewidth=120)

## Loading Our Fashion MNIST Dataset

In [2]:
train_set=datasets.FashionMNIST(root='',download=True,train=True,transform=transforms.Compose([transforms.ToTensor()]))

Extracting FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST/raw


Extracting FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST/raw


Extracting FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST/raw




Extracting FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [3]:
# We will Define our model in this class
class Network(nn.Module): 
  def __init__(self):
    super().__init__()
    # Defining Layers we will use
    # conv2d is CNN layer
    self.con1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
    self.con2=nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
    # Linear is fully connected layer can be termed as a basic neural network layer
    self.fc1=nn.Linear(in_features=12*4*4,out_features=120)
    self.fc2=nn.Linear(in_features=120,out_features=60)
    self.out=nn.Linear(in_features=60,out_features=10)
  def forward(self,t):
    # Defining Layer Architecture
    t=t
    t=self.con1(t)
    # relu is activation function which returns the value as it is if it is greater than or equls to zero else it returns zero.
    t=F.relu(t)
    # max_pool2d is a max pooling layer which pools the max value out of image since kernel size is 2 
    # it will convolve on image and will take max value from 2*2 matrix of image.
    t=F.max_pool2d(t,kernel_size=2,stride=2)
    t=self.con2(t)
    t=F.relu(t)
    t=F.max_pool2d(t,kernel_size=2,stride=2)
    # reshape is used to convert our output such that it can be fed to next layer by flattening the output
    t=t.reshape(-1,12*4*4)
    t=self.fc1(t)
    t=F.relu(t)
    t=self.fc2(t)
    t=F.relu(t)
    t=self.out(t)
    return t


In [17]:
# this function is used to return total no. of correct prediction made by our model
def get(preds,labels):
  return preds.argmax(dim=1).eq(labels).sum().item()


In [15]:
network=Network()
train_loader=torch.utils.data.DataLoader(train_set,batch_size=64,shuffle=True)
optimis=optim.Adam(network.parameters(),lr=0.05)
for epoch in range(5):
    total_correct=0
    total_loss=0
    for batch in train_loader:
      images,labels=batch
      preds=network(images)
      optimis.zero_grad()
      loss=F.cross_entropy(preds,labels)
      loss.backward()
      optimis.step()
      total_loss+=loss.item()
      total_correct+=get(preds,labels)
    print('epoch:',epoch,'correct:',total_correct,"loss:",total_loss)  


epoch: 0 correct: 40224 loss: 797.2905095219612
epoch: 1 correct: 43905 loss: 682.0779508650303
epoch: 2 correct: 43771 loss: 695.6711677014828
epoch: 3 correct: 43557 loss: 710.3963221907616
epoch: 4 correct: 44270 loss: 683.8600843846798


### Below code can be used to initiate tensorboard for Hyperperameter Tuning in google Colab
#### %load_ext tensorboard
#### %tensorboard --logdir=runs


### for individuals who are working on there own system can go to there anaconda prompt or cmd where pytorch is installed and run this command
#### tensorboard --logdir=runs

Tensorbord Can be used for Hyper Perameter Tuning(Perameter Which are set by users like learning, batch size, optimizers) to increase the model accuracy.